Importando as bibliotecas necessárias

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
import time
import os
from bs4 import BeautifulSoup
import csv
import os
import pandas as pd
import numpy as np

RANDOM_SEED = 42

Extração dos urls das páginas com os vídeos e estatísticas das partidas

In [2]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

url = 'https://lolesports.com/pt-BR/vods/cblol-brazil/cblol_split_1_2024'
driver.get(url)

time.sleep(5)

video_url = []

base_xpath = '//*[@id="vodsRoot"]/div[2]/div[1]/div[{}]/div/div[2]/div[1]/div[{}]/*'

i, j = 2, 1 
tentativas = 0  

while True:
    try:
        button_xpath = base_xpath.format(i, j)
        button = driver.find_element(By.XPATH, button_xpath)
        video_url.append(button.get_attribute('href'))
        j += 1
        tentativas = 0
    except:
        if tentativas < 2:
            i += 1
            j = 1  
            tentativas += 1
        else:
            break

# Fecha o navegador
driver.quit()

In [4]:
len(video_url)

140

Função para extrair o html de cada jogo, percorrendo a lista de urls e salvando o tempo da partida de acordo com a parte do vídeo (early, mid e late game):

In [5]:
# Get the current working directory
current_directory = os.getcwd()

driver = webdriver.Chrome()
count = 1

def get_game_data(driver, count, game_time):
        #switch case to game_time ("early", "mid", "late")
        if game_time == "early":
            actions.send_keys(Keys.NUMPAD4).perform()
            time.sleep(8)
        elif game_time == "mid":
            actions.send_keys(Keys.NUMPAD6).perform()
            time.sleep(8)
        elif game_time == "late":
            actions.send_keys(Keys.NUMPAD9).perform()
            time.sleep(8)

        #get the htlm of the page
        html = driver.page_source

        file_name = f"{game_time}_lolesports_{count}.txt"
        # Obter o diretório do script atual
        script_directory = os.path.dirname(os.path.abspath(current_directory))
        
        # Construir o caminho completo para o arquivo 'lolesports.txt' dentro da pasta 'data/raw'
        file_path = os.path.join(script_directory, 'data', 'raw', file_name)
        # Garantir que o diretório 'data/raw' exista
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        
        # Salvar a saída no arquivo
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(html)

for url in video_url:
    try:
        driver.get(url)
        
        actions = ActionChains(driver)

        print("Waiting for video to load")
        time.sleep(5)
        print("Video loaded")

        start_button = driver.find_element(By.XPATH, "//*[@class='attached']")
        start_button.click()

        time.sleep(5)

        get_game_data(driver, count, "early")
        get_game_data(driver, count, "mid")
        get_game_data(driver, count, "late")
        
        count += 1
    except Exception as e:
        print(e)
        
print("Closing driver")
driver.quit()



Waiting for video to load
Video loaded
Waiting for video to load
Video loaded
Waiting for video to load
Video loaded
Waiting for video to load
Video loaded
Waiting for video to load
Video loaded
Waiting for video to load
Video loaded
Waiting for video to load
Video loaded
Waiting for video to load
Video loaded
Waiting for video to load
Video loaded
Message: invalid argument: 'url' must be a string
  (Session info: chrome=125.0.6422.76)
Stacktrace:
	GetHandleVerifier [0x00007FF753D81F52+60322]
	(No symbol) [0x00007FF753CFCEC9]
	(No symbol) [0x00007FF753BB7EBA]
	(No symbol) [0x00007FF753C4C6C4]
	(No symbol) [0x00007FF753C2C21A]
	(No symbol) [0x00007FF753C4BC80]
	(No symbol) [0x00007FF753C2BFC3]
	(No symbol) [0x00007FF753BF9617]
	(No symbol) [0x00007FF753BFA211]
	GetHandleVerifier [0x00007FF75409949D+3301613]
	GetHandleVerifier [0x00007FF7540E36C3+3605267]
	GetHandleVerifier [0x00007FF7540D9440+3563664]
	GetHandleVerifier [0x00007FF753E34326+790390]
	(No symbol) [0x00007FF753D0750F]
	(No 

In [8]:
# Obter o diretório do script atual
script_directory = os.path.dirname(os.path.abspath(current_directory))

def get_html_file(file_name):
    # Construir o caminho completo para o arquivo 'lolesports.txt' dentro da pasta 'data/raw'
    file_path = os.path.join(script_directory, 'data', 'raw', file_name)

    # Abrir o arquivo
    with open(file_path, 'r', encoding='utf-8') as file:
        html = file.read()
    
    return html

def get_lane_players_stats(soup, lane):

    if lane == 'top':
        # Tratamento especial devido as classes obtidas como o crawler
        lane_players = soup.find_all('div', {'class': f'player top selected primary'})
        lane_players += soup.find_all('div', {'class': f'player top selected primary dead'})
        lane_players += soup.find_all('div', {'class': f'player top selected secondary'})
        lane_players += soup.find_all('div', {'class': f'player top selected secondary dead'})

    else:
        # Encontrar todos os elementos com a classe 'player {lane}' para ambos os times
        lane_players = soup.find_all('div', {'class': f'player {lane}'})

        # Encontrar todos os elementos com a classe 'player {lane} dead' para ambos os time
        lane_players += soup.find_all('div', {'class': f'player {lane} dead'})

    # Iterar sobre os jogadores encontrados
    player_stats = []
    for lane_player in lane_players:
        # Obter o nome do jogador
        player_name = lane_player.find('div', {'class': 'name'}).text
        player_name = player_name.split(" ")[1]

        # Obter as estatísticas de kills, deaths e assists
        kda_element = lane_player.find('div', {'class': 'stat kda'})
        kills = kda_element.find('span', {'class': 'kills'}).text
        deaths = kda_element.find('span', {'class': 'deaths'}).text
        assists = kda_element.find('span', {'class': 'assists'}).text

        # Obter as estatísticas de cs e gold
        cs_element = lane_player.find('div', {'class': 'stat cs'})
        cs = cs_element.text

        gold_element = lane_player.find('div', {'class': 'stat gold'})
        gold = gold_element.text

        player_stats = [player_name, kills, deaths, assists, cs, gold]
        # print(f"{player_name} KDA: {kills}/{deaths}/{assists} CS: {cs} Gold: {gold}")

    return player_stats

def get_team_name(soup):
    lane_players = soup.find_all('div', {'class': f'player mid'})
    lane_players += soup.find_all('div', {'class': f'player mid dead'})

    for lane_player in lane_players:
        # Obter o nome do jogador
        player_name = lane_player.find('div', {'class': 'name'}).text

        team_name = player_name.split(" ")[0]

    return team_name

lanes = ['top', 'jungle', 'mid', 'bottom', 'support']
# lanes = ['bottom']

# Construir o caminho completo para o arquivo 'data.csv' dentro da pasta 'data/processed'
file_path = os.path.join(script_directory, 'data', 'processed', 'data.csv')

# Garantir que o diretório 'data/processed' exista
os.makedirs(os.path.dirname(file_path), exist_ok=True)

# Create a cvs file to save the data
with open(file_path, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["game","game_time",
                     "blue_team", "blue_team_dragons", "blue_team_gold", "blue_team_inhibitors", "blue_team_barons", "blue_team_towers", "blue_team_kills",
                     "blue_team_top", "blue_team_top_kills", "blue_team_top_deaths", "blue_team_top_assists", "blue_team_top_cs", "blue_team_top_gold",
                     "blue_team_jungle", "blue_team_jungle_kills", "blue_team_jungle_deaths", "blue_team_jungle_assists", "blue_team_jungle_cs", "blue_team_jungle_gold",
                     "blue_team_mid", "blue_team_mid_kills", "blue_team_mid_deaths", "blue_team_mid_assists", "blue_team_mid_cs", "blue_team_mid_gold",
                     "blue_team_bottom", "blue_team_bottom_kills", "blue_team_bottom_deaths", "blue_team_bottom_assists", "blue_team_bottom_cs", "blue_team_bottom_gold",
                     "blue_team_support", "blue_team_support_kills", "blue_team_support_deaths", "blue_team_support_assists", "blue_team_support_cs", "blue_team_support_gold",
                     "red_team", "red_team_dragons", "red_team_gold", "red_team_inhibitors", "red_team_barons", "red_team_towers", "red_team_kills",
                     "red_team_top", "red_team_top_kills", "red_team_top_deaths", "red_team_top_assists", "red_team_top_cs", "red_team_top_gold",
                     "red_team_jungle", "red_team_jungle_kills", "red_team_jungle_deaths", "red_team_jungle_assists", "red_team_jungle_cs", "red_team_jungle_gold",
                     "red_team_mid", "red_team_mid_kills", "red_team_mid_deaths", "red_team_mid_assists", "red_team_mid_cs", "red_team_mid_gold",
                     "red_team_bottom", "red_team_bottom_kills", "red_team_bottom_deaths", "red_team_bottom_assists", "red_team_bottom_cs", "red_team_bottom_gold",
                     "red_team_support", "red_team_support_kills", "red_team_support_deaths", "red_team_support_assists", "red_team_support_cs", "red_team_support_gold"
                     ])

#ler cada arquivo da pasta raw
# files = ["late_lolesports_1.txt"]
for file_name in os.listdir(os.path.join(script_directory, 'data', 'raw')):
# for file_name in files:
    # Ignorar arquivos que não terminam com '.txt'
    if not file_name.endswith('.txt'):
        continue

    # Obter o html do arquivo
    html = get_html_file(file_name)

    # Criar o objeto BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # print(f"Arquivo: {file_name}")
    # print(" ")
    game_time = file_name.split("_")[0]

    stats_team_summary = soup.find('div', {'class': 'StatsTeamsSummary'})
    if stats_team_summary is None:
        continue
    
    # Get dragons stats
    dragons = stats_team_summary.find('div', {'class': 'dragons'})

    blue_team_dragons = dragons.find_all('div', {'class': 'blue-team'})
    red_team_dragons = dragons.find_all('div', {'class': 'red-team'})

    blue_team_dragons_list = []
    for blue_team_dragon in blue_team_dragons:

        dragon_divs = blue_team_dragon.find_all('div')

        for div in dragon_divs:
            if 'class' in div.attrs:
                classes = div['class']
                blue_team_dragons_list.append(classes[1])
    
    red_team_dragons_list = []
    for red_team_dragon in red_team_dragons:

        dragon_divs = red_team_dragon.find_all('div')

        for div in dragon_divs:
            if 'class' in div.attrs:
                classes = div['class']
                red_team_dragons_list.append(classes[1])
    
    # Get gold stats
    gold = stats_team_summary.find('div', {'class': 'gold'})
    totals = gold.find('div', {'class': 'totals'})
    blue_team_gold = totals.find('div', {'class': 'blue-team'}).text
    red_team_gold = totals.find('div', {'class': 'red-team'}).text

    # Get details
    details = stats_team_summary.find('div', {'class': 'details'})
    blue_team_details = details.find('div', {'class': 'blue-team'})
    blue_team_inhibitors = blue_team_details.find('div', {'class': 'stat inhibitors'}).text
    blue_team_barons = blue_team_details.find('div', {'class': 'stat barons'}).text
    blue_team_towers = blue_team_details.find('div', {'class': 'stat towers'}).text
    blue_team_kills = blue_team_details.find('div', {'class': 'stat kills'}).text

    red_team_details = details.find('div', {'class': 'red-team'})
    red_team_inhibitors = red_team_details.find('div', {'class': 'stat inhibitors'}).text
    red_team_barons = red_team_details.find('div', {'class': 'stat barons'}).text
    red_team_towers = red_team_details.find('div', {'class': 'stat towers'}).text
    red_team_kills = red_team_details.find('div', {'class': 'stat kills'}).text

    stats_team_playes = soup.find('div', {'class': 'StatsTeamsPlayers'})

    blue_team = stats_team_playes.find('div', {'class': 'blue-team'})
    blue_team_name = get_team_name(blue_team)
    blue_team_players_stats = []
    for lane in lanes:
        blue_team_players_stats += get_lane_players_stats(blue_team, lane)

    red_team = stats_team_playes.find('div', {'class': 'red-team'})
    red_team_name = get_team_name(red_team)

    red_team_players_stats = []
    for lane in lanes:
        red_team_players_stats += get_lane_players_stats(red_team, lane)
    game_id = file_name.split("_")[2].split(".")[0]
    
    # Write the data in the csv file
    with open(file_path, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([game_id, game_time,
                         blue_team_name, blue_team_dragons_list, blue_team_gold, blue_team_inhibitors, blue_team_barons, blue_team_towers, blue_team_kills] + blue_team_players_stats +
                         [red_team_name, red_team_dragons_list, red_team_gold, red_team_inhibitors, red_team_barons, red_team_towers, red_team_kills] + red_team_players_stats
                        )

In [10]:
# Obtendo o diretório atual do notebook
notebook_dir = %pwd

# Construindo o caminho completo para data.csv
file_path = os.path.join(notebook_dir, '..', 'data', 'processed', 'data.csv')

In [11]:
# Lendo o arquivo CSV usando pandas
data_raw = pd.read_csv(file_path)

data = data_raw.copy()

#Exibe as 5 primeiras linhas do dataframe
data.head(12)

,game,game_time,blue_team,blue_team_dragons,blue_team_gold,blue_team_inhibitors,blue_team_barons,blue_team_towers,blue_team_kills,blue_team_top,...,red_team_bottom_deaths,red_team_bottom_assists,red_team_bottom_cs,red_team_bottom_gold,red_team_support,red_team_support_kills,red_team_support_deaths,red_team_support_assists,red_team_support_cs,red_team_support_gold
0,1,early,LLL,[],0,0,0,0,0,Robo,...,0,0,0,0,Kuri,0,0,0,0,0
1,10,early,VKS,[],21.6 K,0,0,0,3,Guigo,...,1,1,121,4.8 K,JoJo,0,1,2,15,2.9 K
2,100,early,KBM,[],2.5 K,0,0,0,0,Lonely,...,0,0,0,500,Damage,0,0,0,0,500
3,101,early,RED,"['cloud', 'infernal']",22.0 K,0,0,0,5,fNb,...,0,1,26,2.9 K,scamber,0,1,0,90,3.9 K
4,102,early,KBM,[],2.5 K,0,0,0,0,Lonely,...,0,0,0,500,Kuri,0,0,0,0,500
5,103,early,ITZ,[],2.5 K,0,0,0,0,Tay,...,0,0,0,500,RedBert,0,0,0,0,500
6,104,early,LOS,[],18.8 K,0,0,0,4,SuperCleber,...,1,0,112,5.0 K,zay,0,1,3,21,3.1 K
7,105,early,VKS,[],2.5 K,0,0,0,0,Guigo,...,0,0,0,500,Cavalo,0,0,0,0,500
8,106,early,ITZ,['ocean'],18.6 K,0,0,0,0,Tay,...,0,0,93,3.6 K,zay,0,0,0,16,3.1 K
9,107,early,LLL,['cloud'],15.4 K,0,0,0,0,Robo,...,0,0,82,3.2 K,scamber,0,0,0,19,2.2 K


In [13]:
data.shape

(377, 76)